# FEMA ETL Process

The city needs to consume data from FEMA into their own databases for purposes of custom visualization and information tracking. This script utilizes conda and ESRI's `arcgis` python API to draw information from REST End Points down into the enterprise SDE

In [ ]:
from arcgis.features import FeatureLayer
from arcgis.features import FeatureLayerCollection
from arcgis.geometry import filters
import datetime
import re

Convert relevant endpoint URLs into ESRI FeatureLayer objects

In [ ]:
city_lims_url = "https://maps.bouldercolorado.gov/arcgis/rest/services/plan/CityLimits/MapServer/0"
city = FeatureLayer(city_lims_url)

nfhl_url = "https://hazards.fema.gov/gis/nfhl/rest/services/public/NFHL/MapServer"
nfhl = FeatureLayerCollection(nfhl_url)
lomr = nfhl.layers[1]
loma = nfhl.layers[2]
firm = nfhl.layers[3]
xs = nfhl.layers[14]
sfha = nfhl.layers[28]

Define the ouput spatial reference

In [ ]:
sr = 2876 # NAD83(HARN) / Colorado North (ftUS)

Extract city limits as a spatial filter (We want to be able to check that LOMRs happened inside city limits)

In [ ]:
city_ext = city.properties.extent
geom_filter = filters.contains(city_ext)

## LOMRs

In [ ]:
boulder_lomrs = lomr.query(geometry_filter=geom_filter, out_sr=sr, as_df=True)
boulder_lomrs.sort_values(['EFF_DATE'])
# boulder_lomrs.features[0].attributes

Find the most recent update to NFHL floodplains in Boulder

In [ ]:
# returns as a timestamp
most_recent_lomr = boulder_lomrs.EFF_DATE.max()
most_recent_lomr

## LOMAs

Based on a little bit of data exploration, we know that the City of Boulder is specifically called out in the COMMUNITYNAME field as "Boulder, City of". There are 235 LOMAs listed inside the city, however, their geolocations were obtained based on a national-scale geocoder, and they're not very accurate as a result.

In [ ]:
where_statement = "COMMUNITYNAME = 'Boulder, City of'"
boulder_lomas = loma.query(where=where_statement, out_sr=sr, as_df=True)
boulder_lomas

In [ ]:
re.findall("(.*) +-+ +(.*)", boulder_lomas.PROJECTNAME[5])

In [ ]:
boulder_lomas.iloc[6]